Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19880 sha256=43fea99a7dbafd6900cf52584d67fc0cefe652912f5a036cf263b5b7eb41982b
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-07-31 18:11:07,951 INFO numerapi.utils: starting download
./numerai_dataset_275.zip: 100%|█████████▉| 415M/415M [00:37<00:00, 17.8MB/s]2021-07-31 18:11:45,040 INFO numerapi.base_api: unzipping file...
./numerai_dataset_275.zip: 415MB [00:49, 17.8MB/s]                           

In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 59.9 s, sys: 10.8 s, total: 1min 10s
Wall time: 1min 12s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
# Xgboost Built-in Feature Importance

In [ ]:
pip install shap

     |████████████████████████████████| 356 kB 13.8 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491649 sha256=61c4e9173dfba4666aa22054266414c792658a9219cf472c1a5340b04508076b
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [ ]:
import numpy as np
import pandas as pd
import shap

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
import seaborn as sns # for correlation heatmap

from xgboost import XGBRegressor

In [ ]:
X_data = training_data[feature_names]
y_data = training_data[training_data.columns[-1]]

In [ ]:
# Split data into train and test samples (random ordering)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, train_size=0.8)

In [ ]:
xgb = XGBRegressor(n_estimators=100)
xgb.fit(X_train, y_train)

[18:15:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
feature_importances=pd.DataFrame({'features':feature_names,'feature_importance':xgb.feature_importances_})
print(feature_importances.sort_values('feature_importance',ascending=False))

                    features  feature_importance
29        feature_charisma18            0.012645
253  feature_constitution104            0.012384
87        feature_charisma76            0.010659
66        feature_charisma55            0.009492
174   feature_constitution25            0.008903
..                       ...                 ...
166   feature_constitution17            0.000000
168   feature_constitution19            0.000000
169   feature_constitution20            0.000000
171   feature_constitution22            0.000000
309         feature_wisdom46            0.000000

[310 rows x 2 columns]


In [ ]:
feature_importances_best = feature_importances[feature_importances.feature_importance > 0.0025]

In [ ]:
feature_names = feature_importances_best['features'].values

In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
%%time

model_e = LGBMRegressor(random_state=rand)
gc.collect()

model = LGBMRegressor(random_state=rand)
model.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

y_pred_model = model.predict(tournament_data[feature_names])
gc.collect()

tournament_data[PREDICTION_NAME] = y_pred_model


CPU times: user 1min 3s, sys: 6.64 s, total: 1min 10s
Wall time: 1min 10s


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.491713
n000920ed083903f    0.490011
n0038e640522c4a6    0.518279
n004ac94a87dc54b    0.504401
n0052fe97ea0c05f    0.501863
                      ...   
nffd112483053b7a    0.483455
nffda23fd6297b41    0.479363
nffed587b72309c4    0.497859
nffeda441259a613    0.484864
nfff3b39f846a436    0.503654
Name: prediction, Length: 1779480, dtype: float64

In [ ]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


In [ ]:
%%time
tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

CPU times: user 2min 13s, sys: 23.1 s, total: 2min 36s
Wall time: 55.3 s


In [ ]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.535869
n000920ed083903f    0.490862
n0038e640522c4a6    0.510686
n004ac94a87dc54b    0.557182
n0052fe97ea0c05f    0.536793
                      ...   
nffd112483053b7a    0.409009
nffda23fd6297b41    0.524374
nffed587b72309c4    0.499684
nffeda441259a613    0.459864
nfff3b39f846a436    0.490558
Name: prediction, Length: 1779480, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol_0_b.csv')

!cp FisaGol_0_b.csv "drive/My Drive/PHOENIXSIGMA/"